# Data pre-processing for final website

This small notebook is used to prepare the data for the final visualization : essentially, aggregating them and putting them in the correct format.

In [11]:
import pandas as pd
import unidecode
import re
import math
import locale
import matplotlib.pyplot as plt
import numpy as np
import random
import json

In [12]:
def open_list(filename):
    names = []
    with open(filename, 'r') as fp:
        for line in fp:
            names.append(line[:-1])
        return names

### Data cleaning

Took the same cleaning code used for milestone 1

In [13]:
# Helper function
def format_username(username):
    if not ', ' in username:
        return username
    words = re.findall('[A-Za-z0-9][^A-Z]*', username)
    return ''.join(words[:-2])

In [14]:
# Load dataset
users = pd.read_csv('website/data/users.csv')

# To remove every non-ASCII character
users = users.applymap(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

# Convert registration date string to proper year
users['registration_date'] = users['registration_date'].str.replace('Membre Tripadvisor depuis ', '')
users['registration_date'] = users['registration_date'].str.replace('None', '-1')
users['registration_date'] = users['registration_date'].astype(int)

# Convert number-of-comments string int an int value
def format_error(row):
    if 'ville' in row['nb_of_comments']:
        row['nb_of_cities'] = row['nb_of_comments']
        row['nb_of_comments'] = '0'
    if 'vote' in row['nb_of_cities']:
        row['nb_of_comments'] = row['nb_of_cities']
        row['nb_of_cities'] = '0'
    if 'vote' in row['nb_of_comments']:
        row['nb_of_comments'] = row['nb_of_comments'].replace(' votes utiles', '')
        row['nb_of_comments'] = row['nb_of_comments'].replace(' vote utile', '')
    if 'photo' in row['nb_of_cities']:
        row['nb_of_cities'] = '0'
        
    return row

users['nb_of_comments'] = users['nb_of_comments'].str.replace('contributions', '')
users['nb_of_comments'] = users['nb_of_comments'].str.replace('contribution', '')
users['nb_of_comments'] = users['nb_of_comments'].str.replace('None', '0')
users['nb_of_comments'] = users['nb_of_comments'].apply(unidecode.unidecode)
users = users.apply(format_error, axis=1)
users['nb_of_comments'] = users['nb_of_comments'].str.replace(' ', '')
users['nb_of_comments']= users['nb_of_comments'].astype(int)

# Convert the number-of-cities string into an int value
users['nb_of_cities'] = users['nb_of_cities'].str.replace('ville visitee', '')
users['nb_of_cities'] = users['nb_of_cities'].str.replace('villes visitees' ,'')
users['nb_of_cities'] = users['nb_of_cities'].str.replace('None', '0')
users['nb_of_cities'] = users['nb_of_cities'].str.replace(' ', '')
users['nb_of_cities'] = users['nb_of_cities'].astype(int)

users['sex'] = users['sex'].str.replace('homme', 'H')
users['sex'] = users['sex'].str.replace('femme', 'F')
users['username'] = users['username'].apply(format_username)


def format_cuisine_occur(cuisines):    
    if isinstance(cuisines, float) and math.isnan(cuisines):
        return {}
    
    cuisines_dict = {}
    pattern = '\d+ +(\w+) +(\d+)'
    matches = re.findall(pattern, cuisines)   
    
    for match in matches:
        if match[0] != 'None':
            cuisines_dict[match[0]] = int(match[1])
    return cuisines_dict

users['top_cuisines'] = users['top_cuisines'].apply(format_cuisine_occur)

users = users[['username', 'registration_date', 'nb_of_comments', 'nb_of_cities', 'age', 'sex', 'country', 'top_cuisines', 'avg_overall_rating', 'avg_cuisine_rating', 'avg_service_rating', 'avg_quality_price_rating', 'avg_mood_rating', 'avg_michelin', 'avg_price_range_estimation']]
users.head()

,username,registration_date,nb_of_comments,nb_of_cities,age,sex,country,top_cuisines,avg_overall_rating,avg_cuisine_rating,avg_service_rating,avg_quality_price_rating,avg_mood_rating,avg_michelin,avg_price_range_estimation
0,nathalieval1972,2021,10,2,None,None,None,"{'Restauration_rapide': 3, 'Francaise': 2, 'Eu...",3.64,2.74,2.67,2.73,0.57,0.0,1.31
1,johanm8,2018,3,3,None,None,None,{},4.50,0.00,0.00,0.00,0.00,0.0,0.00
2,DumasDD,2013,23,14,50-64,H,france,"{'Europeenne': 2, 'Suisse': 1, 'Francaise': 1,...",4.17,2.67,2.50,2.50,1.33,0.0,1.00
3,ernestbenoit74,2016,35,22,None,None,france,"{'Mediterraneenne': 2, 'Italienne': 2, 'Europe...",4.00,2.05,2.05,1.78,1.50,0.0,1.30
4,cerisecherry,2011,64,40,35-49,F,france,"{'Francaise': 10, 'Europeenne': 10, 'Italienne...",4.11,3.90,3.84,3.63,2.18,0.0,1.78


In [17]:
restaurants = pd.read_csv('website/data/restaurants.csv')
restaurants = restaurants.applymap(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

relevant_meals = ['Diner', 'Petit dejeuner', 'Dejeuner']

def format_food_details(data, relevant, irrelevant):
    food = data.split(', ')
    filtered_food = [x for x in food if x not in irrelevant]
    if len(relevant) != 0:
        filtered_food = list(set(food).intersection(relevant))
    return filtered_food

restaurants['meal_types'] = restaurants['meal_types'].apply(lambda x : format_food_details(x, relevant_meals, []))
restaurants['cuisines'] = restaurants['cuisines'].apply(lambda x : format_food_details(x, [], ['0', 'None']))
restaurants['veggie'] = restaurants['special_regimes'].str.contains("Vegetariens")
restaurants = restaurants.rename(columns={'restaurant_name': 'name'})
restaurants = restaurants[['name', 'adress', 'city_ranking', 'overall_rating', 'number_of_reviews', 'price_range_estimation', 'veggie', 'meal_types', 'cuisines', 'cuisine_rating', 'service_rating', 'quality_price_rating', 'mood_rating', 'michelin']]
restaurants.head()

,name,adress,city_ranking,overall_rating,number_of_reviews,price_range_estimation,veggie,meal_types,cuisines,cuisine_rating,service_rating,quality_price_rating,mood_rating,michelin
0,Bistro Spatz,"Dorfstrasse 2, Freienstein-Teufen 8427 Suisse",7.0,5.0,12,2,False,"[Dejeuner, Diner, Petit dejeuner]","[Suisse, Europeenne]",5.0,0.0,4.0,0.0,0
1,Wirtshus zum Wyberg,"Oberteufenerstrasse 1, Freienstein-Teufen 8428...",2.0,4.5,66,3,True,"[Diner, Dejeuner]","[Suisse, Europeenne, Europe centrale]",4.5,4.5,4.0,4.0,1
2,Alpengasthof Tibethuette,"Stilfser Joch, 39020, Stelvio Italie",33.0,4.5,3,0,False,[],[Italie du Nord],0.0,0.0,0.0,0.0,0
3,Tibet Hutte,"Passo Stelvio, 39029, Stelvio Italie",21.0,4.0,109,2,True,"[Dejeuner, Diner, Petit dejeuner]","[Italienne, Bar, Europeenne, Pub]",4.0,4.0,3.5,4.5,0
4,Restaurant Gallia,"Gomagoi 18, 39029, Stelvio Italie",18.0,4.5,79,2,True,"[Dejeuner, Diner, Petit dejeuner]","[Internationale, Italienne, Mediterraneenne]",5.0,4.5,4.5,0.0,0


In [7]:
reviews = pd.read_csv('website/data/reviews.csv')
reviews['username'] = reviews['username'].apply(format_username)
locale.setlocale(locale.LC_TIME, 'fr_FR')
reviews['visit_date'] = reviews['visit_date'].str.replace('None', 'janvier 1900')
reviews['review_date'] = reviews['review_date'].str.replace('None', 'janvier 1900')

reviews['review_date'] = pd.to_datetime(reviews['review_date'], format='%d %B %Y')
reviews['visit_date'] = pd.to_datetime(reviews['visit_date'], format='%B %Y')
reviews = reviews[['restaurant_name', 'username', 'rating', 'review_date', 'visit_date']]
reviews.head()

,restaurant_name,username,rating,review_date,visit_date
0,Bistro Spatz,maximiuliana,5,2021-08-19,2021-07-01
1,Bistro Spatz,gedeonberger,5,2022-07-10,2022-07-01
2,Bistro Spatz,Ritschi64,5,2022-06-14,2022-06-01
3,Bistro Spatz,DomeH20,5,2022-02-04,2022-02-01
4,Bistro Spatz,Ritschi64,5,2021-11-03,2021-11-01


### Get the list of all possible cuisines

In [18]:
df_exploded = restaurants.explode('cuisines')
cuisines = set(df_exploded['cuisines'].unique())
## remove nan values
cuisines = {x for x in cuisines if x==x}
print(cuisines)

{'Asiatique', 'Sri lankaise', 'Britannique', 'Balti', 'Mongole', 'Thai', 'cambodgienne', 'Portugaise', 'Catalane', 'Hollandaise', 'Europe centrale', 'Caraibes', 'Suedoise', 'Xinjiang', 'Cubaine', 'Pub', 'Internationale', 'Marocaine', 'Ecossaise', 'Chinoise', 'Libanaise', 'Georgienne', 'Fusion japonaise', 'Canadienne', 'Cajun & Creole', 'Restaurants de biere', 'Japonaise', 'Jamaicaine', 'Ukrainienne', 'Indienne', 'Moderne', 'Diner', 'Sud-Ouest americain', 'Bangladaise', 'Saine', 'Sud-americaine', 'Bar a bieres', 'Fusion', 'Roumaine', 'Grecque', 'Americaine', 'Perse', 'Assyrien', 'Mediterraneenne', 'Tcheque', "Europe de l'Est", 'Romana', 'Israelienne', 'Kaiseki', 'Barbecue/Grillades', 'Hongroise', 'Cuisine de Pekin', 'Egyptienne', 'Soupes', 'Salvadorienne', 'Afghane', 'Equatorienne', 'Autrichienne', 'Italienne', 'Pizza', 'Italie du Centre', 'Polynesienne', 'Latium', 'Tunisienne', 'Africaine', 'Bar', 'Neo-zelandaise ', 'Belge', 'Francaise', 'Norvegienne', 'Magasins de fruits', 'Calabraise

In [19]:
# Compute the restaurants serving each cuisine
restaurants_per_cuisine = {}
for c in cuisines :
    filtered_df = restaurants[restaurants['cuisines'].apply(lambda x: c in x)]  
    restaurants_per_cuisine[c] = filtered_df['name'].tolist()

### Getting all the reviews per cuisine

In [9]:
reviews_per_cuisine = {}
for c in cuisines :
    c_reviews = reviews[reviews['restaurant_name'].apply(lambda x: x in restaurants_per_cuisine[c])][['rating','review_date']]
    reviews_per_cuisine[c] = c_reviews

In [10]:
# Get the 20 cuisines with the most reviews
top20 = sorted(reviews_per_cuisine.items(), key=lambda x: len(x[1]), reverse=True)
top20 = dict(top20[:20])
for c in top20:
    print(c, len(top20[c]))


Europeenne 297522
Francaise 182231
Italienne 158226
Suisse 138210
Mediterraneenne 97212
Pizza 73720
Europe centrale 50598
Asiatique 47169
Bar 37025
Fruits de mer & Poisson 31950
Internationale 29788
Italie du Nord 27835
Lombarde 26197
Cafe 23938
Saine 23722
Americaine 21471
Pub 18756
Japonaise 16904
Chinoise 16856
Barbecue/Grillades 16410


In [11]:
# Export list of cuisines
top20_cuisines = list(top20.keys())
with open('statistics/data/top20_cuisine.txt', 'w') as fp:
    for item in top20_cuisines:
        # write each item on a new line
        item = item.replace("/", "_" )
        item = item.replace(" ", "_" )
        fp.write("%s\n" % item)
    print('Top 20 exported')

Top 20 exported


In [ ]:
# Export reviews per cuisine
for c in top20 :
    cuisine_ratings = top20[c]
    c = c.replace("/", "_" )
    c = c.replace(" ", "_" )
    cuisine_ratings.to_csv(f'statistics/data/cuisine_reviews/raw/{c}_reviews', index=True, header=True)
print("Reviews exported")

In [ ]:
# Export reviews per cuisine
for c in top20 :
    df = top20[c].copy(deep=True)
    df.set_index('review_date', inplace=True)
    grouped_df = df.resample('M').agg({'rating': ['count', 'mean']})
    grouped_df.columns = ['count', 'avg']
    grouped_df['avg'].fillna(0, inplace=True)
    c = c.replace("/", "_" )
    c = c.replace(" ", "_" )
    grouped_df.reset_index(inplace=True)
    grouped_df['review_date'] = grouped_df['review_date'].dt.strftime('%Y-%m-%d')
    grouped_df.to_json(f'statistics/data/cuisine_reviews/aggregated/{c}_reviews', orient='records')
print("Reviews exported")

### Get reviewers per cuisine

In [ ]:
reviewers_per_cuisine = {}
for c in top20:
    if "/" not in c:
        continue
    c_reviewers = reviews[reviews['restaurant_name'].apply(lambda x: x in restaurants_per_cuisine[c])]['username'].unique()
    print("unique reviewers : ", len(c_reviewers))   
    c = c.replace("/", "_" )
    c = c.replace(" ", "_" )
    with open(f'statistics/data/cuisine_users/simple/{c}_users.txt', 'w') as fp:
        for item in c_reviewers:
            # write each item on a new line
            fp.write("%s\n" % item)
print('raw reviewers exported')

In [ ]:
for c in top20:
    c = c.replace("/", "_" )
    c = c.replace(" ", "_" )
    names = []
    with open(f'statistics/data/cuisine_users/simple/{c}_users.txt', 'r') as fp:
        for line in fp:
            names.append(line[:-1])
    print(len(names))
    users_df = users[users['username'].apply(lambda x: x in names)]
    users_df.to_csv(f'statistics/data/cuisine_users/all/{c}_users.csv')
    print(c + " done")

### Aggregate reviewers data for each cuisine

In [16]:
top20 = open_list('statistics/data/top20_cuisine.txt')

In [ ]:
# I just found out that they might be duplicate in the user csv.
for c in top20:
    df = pd.read_csv(f'statistics/data/cuisine_users/all/{c}_users.csv')
    before = len(df)
    df.drop_duplicates(subset=['username'], keep='first', inplace=True)
    df = df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.to_csv(f'statistics/data/cuisine_users/all/{c}_users.csv', index=False)

### Get the ages distribution of those reviewsers (we left None values out)

In [48]:
for c in top20:
    df = pd.read_csv(f'statistics/data/cuisine_users/all/{c}_users.csv')
    ages = df['age'].value_counts().to_frame()
    ages.reset_index(inplace=True)
    ages = ages[ages['index'] != 'None']
    ages.to_json(f'statistics/data/cuisine_users/age/{c}_ages.json', orient="records")
print("Ages exported")

Ages exported


### Get the #reviews distribution of the reviewers for each cuisine

In [52]:
for c in top20:
    df = pd.read_csv(f'statistics/data/cuisine_users/all/{c}_users.csv')
    df['more than 100'] = df['nb_of_comments'] > 100
    df['between 50 and 100'] = (df['nb_of_comments'] <= 100) & (df['nb_of_comments'] > 50)
    df['between 0 and 50'] = (df['nb_of_comments'] <= 50) & (df['nb_of_comments'] >= 0)
    boolean_sum = df[['more than 100', 'between 50 and 100', 'between 0 and 50']].astype(bool).sum()
    boolean_sum = boolean_sum.to_frame()
    boolean_sum = boolean_sum.reset_index()
    boolean_sum.rename(columns={0: 'count', 'index':'category'}, inplace=True)
    boolean_sum.to_json(f'statistics/data/cuisine_users/comments/{c}_comments.json', orient='records')
print("Comments exported")


Comments exported
